<h1>importing necessary libraries</h1>

In [413]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

<h3>importing dataset</h3>

In [414]:
df = pd.read_csv("Pune_House_Data.csv")

<h3>Keeping the necessary values and droppping the rest values</h3>

In [415]:
df = df.drop(columns=["area_type", "availability", "society"])

<h3>finding the missing values in the dataset and fill them.</h3>

In [416]:
df[["size", "site_location"]] = df[["size", "site_location"]].fillna(method="ffill")
df["bath"] = df["bath"].replace(np.nan, df["bath"].mean())
df["balcony"] = df["balcony"].replace(np.nan, df["balcony"].mean())
df

,size,site_location,total_sqft,bath,balcony,price
0,2 BHK,Alandi Road,1056,2.0,1.000000,39.07
1,4 Bedroom,Ambegaon Budruk,2600,5.0,3.000000,120.00
2,3 BHK,Anandnagar,1440,2.0,3.000000,62.00
3,3 BHK,Aundh,1521,3.0,1.000000,95.00
4,2 BHK,Aundh Road,1200,2.0,1.000000,51.00
...,...,...,...,...,...,...
13315,5 Bedroom,Pashan,3453,4.0,0.000000,231.00
13316,4 BHK,Paud Road,3600,5.0,1.584376,400.00
13317,2 BHK,Pirangut,1141,2.0,1.000000,60.00
13318,4 BHK,Prabhat Road,4689,4.0,1.000000,488.00


<h3>convertng the string data into numeric format to train the model</h3>

In [417]:
# areaString = df["area_type"].unique()
sizeString = df["size"].unique()
locationString = df["site_location"].unique()
sqFeetString = df["total_sqft"].unique()

enc = LabelEncoder()

for col in df.columns[:-3]: 
    df[col] = df[col].astype(str)
    df[col] = enc.fit_transform(df[col])

# areaNumeric = df["area_type"].unique()
sizeNumeric = df["size"].unique()
locationNumeric = df["site_location"].unique()
sqFeetNumeric = df["total_sqft"].unique()



<h3>creating dictionaries to store the values of numeric values and string values respectively</h3>

In [418]:
sizeDict, locationDict = {}, {}

# for key,value in zip(areaNumeric, areaString):
#     areaDict[key] = value

for key,value in zip(sizeNumeric, sizeString):
    sizeDict[key] = value

for key,value in zip(locationNumeric, locationString):
    locationDict[key] = value



<h3>Dividing the dataset into three parts that is train, validation and test .</h3>

In [419]:
train, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])
test

,size,site_location,total_sqft,bath,balcony,price
12955,16,89,511,3.0,0.000000,75.885
2535,16,38,903,3.0,2.000000,165.000
5305,13,24,200,2.0,1.000000,33.640
9946,18,57,1606,4.0,2.000000,345.000
7919,0,46,1784,1.0,0.000000,18.000
...,...,...,...,...,...,...
8473,13,24,2091,2.0,1.000000,56.000
1053,17,93,577,3.0,1.000000,170.000
8806,13,69,57,2.0,2.000000,48.000
5087,16,94,306,3.0,1.584376,63.000


<h3>Scailing the dataset</h3>

In [420]:
def scale_data(dataFrame):
    x = dataFrame[dataFrame.columns[:-1]].values
    y = dataFrame[dataFrame.columns[-1]].values
    # print(type(y))
    scaler = StandardScaler()
    x = scaler.fit_transform(x)
    data = np.hstack((x, np.reshape(y, (-1, 1))))
    return data, x, y
# x = df[df.columns[:-1]].values
# y = df[df.columns[-1]].values
# data = np.hstack((x, np.reshape(y, (-1, 1))))

In [421]:
train, X_train, Y_train = scale_data(train)
validate, X_validate, Y_validate = scale_data(valid)
test, X_test, Y_test = scale_data(test)

# print(test)